In [4]:
from lib.utils import load_raw_by_path
from torch import nn
from torch.nn.functional import batch_norm,relu
from torch import nn
import numpy as np
from torch.nn.functional import softmax
import torch
import sqlite3
from sqlite3 import Error
import os
import pandas as pd
from tqdm import tqdm
EEG_1 = [1,8,14,15,16]
EEG_2 = [3,4,5,6,7,9,10,11,12,13,17]

/home/andrew/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
class MLP(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.fc1 = nn.Linear(15000,128)
        self.fc2 = nn.Linear(128,3)
    def forward(self,x):
        x = self.fc1(x)
        x = relu(x)
        x = self.fc2(x)
        return x

In [8]:
pred = model(X.cuda())

In [13]:
pred = softmax(pred,dim=1).argmax(axis=1)

In [24]:
pred_expert = np.array(pred.detach().cpu()).copy()
print("Experting")
for j in range(len(pred_expert)-2):
    if((pred_expert[j:j+2]==np.array([2,0])).all()):
        pred_expert[j+1] = 2
for j in range(len(pred_expert)-2):
    if(pred_expert[j+1] != pred_expert[j] and pred_expert[j+1] != pred_expert[j+2]):
        pred_expert[j+1] = pred_expert[j]
df = pd.DataFrame(pred_expert)


Experting


In [26]:

print(df[0].value_counts(normalize=True))
df[df[0] == 0] = 'P'
df[df[0] == 1] = 'S'
df[df[0] == 2] = 'W'
df.to_csv(f'pred.csv',index=False)

S    0.604134
W    0.395006
P    0.000860
Name: 0, dtype: float64


In [31]:

model = torch.load('model_1100.pt')
# checkpoint = torch.load('model_106_0.3201.pt')
# model = checkpoint['model_class']
# model.load_state_dict(checkpoint['model_state_dict'])

dists = []
for i in tqdm(range(1,18)):
    if i==2:
        continue
    raw = load_raw_by_path(f'data/aging/22-AGING-{i}.edf').get_data(picks=['EEG','EMG'])
    if(i in EEG_1):
        eeg = raw[0]
    elif(i in EEG_2):
        eeg = raw[1]
    else:
        print("error")
    del raw
    eeg = torch.from_numpy(eeg.reshape(-1,5000))
    X = torch.cat([eeg[:-2],eeg[1:-1],eeg[2:]],axis=1).float()
    print("Centering")
    # eeg_centered = eeg - eeg.mean(axis=1).reshape(-1,1)
    del eeg
    # std = eeg_centered.std(axis=1)
    # std[np.where(std==0)[0]] = 1
    # eeg_norm = eeg_centered/std.reshape(-1,1)
    # del eeg_centered
    # del std
    # X = torch.from_numpy(eeg_norm.reshape(-1,1,5000)).float()
    # del eeg_norm
    # print(X.isinf().any())
    # X = X[:,:,::10]
    y_pred = softmax(model(X.cuda()),dim=1).argmax(axis=1)
    del X
    pred_expert = np.array(y_pred.detach().cpu()).copy()
    del y_pred
    print("Experting")
    for j in range(len(pred_expert)-2):
        if((pred_expert[j:j+2]==np.array([2,0])).all()):
            pred_expert[j+1] = 2
    for j in range(len(pred_expert)-2):
        if(pred_expert[j+1] != pred_expert[j] and pred_expert[j+1] != pred_expert[j+2]):
            pred_expert[j+1] = pred_expert[j]
    df = pd.DataFrame(pred_expert)
    df[df[0] == 0] = 'P'
    df[df[0] == 1] = 'S'
    df[df[0] == 2] = 'W'
    print(df[0].value_counts(normalize=True))
    dists.append(df[0].value_counts(normalize=True))
    df.to_csv(f'data/aging/{i}_pred.csv',index=False)

    offset = 10e7       #epoch time period
    rename_dict = {'W':'Sleep-Wake', 'S':'Sleep-SWS', 'P':'Sleep-Paradoxical', 'X':''}

    csv_filename = f'data/aging/{i}_pred.csv'
    zdb_filename = f'data/aging/22-AGING-{i}.zdb'

    df = pd.read_csv(csv_filename)
    try:
        conn = sqlite3.connect(zdb_filename)
    except Error as e:
        print(e)
    cur = conn.cursor()

    #drop this table - creates issues
    query = "DROP TABLE IF EXISTS temporary_scoring_marker;"
    cur.execute(query)

    #get keyid of scoring
    query = "SELECT MAX(id) FROM scoring_revision WHERE name='Machine Data'"
    cur.execute(query)
    keyid = cur.fetchall()[0][0]

    #get starting point for scoring
    query = "SELECT id FROM scoring_marker WHERE type LIKE 'Sleep%' AND key_id='"+str(keyid)+"';"
    cur.execute(query)
    startid = cur.fetchall()[0][0]

    #get start time to crreate epochs
    query = 'SELECT starts_at FROM scoring_marker WHERE id = '+str(startid)+";"
    cur.execute(query)
    start_time = cur.fetchall()[0][0]
    stop_time = 0

    #delete first score before adding machine data
    query = "DELETE FROM scoring_marker;"
    cur.execute(query)

    #insert new epochs with scoring into the table
    for i in range(len(df)):
        #calculate epoch
        if i != 0:
            start_time = stop_time
        stop_time = start_time+offset

        score = rename_dict[df.at[i,'0']]
        #insert epoch
        query = f"""
                INSERT INTO scoring_marker 
                (starts_at, ends_at, notes, type, location, is_deleted, key_id)
                VALUES 
                ({start_time}, {stop_time}, '', '{score}', '', 0, {keyid});
                """ 
        cur.execute(query)

    conn.commit()
    conn.close()

  0%|          | 0/17 [00:00<?, ?it/s]

Centering
Experting
S    0.604134
W    0.395006
P    0.000860
Name: 0, dtype: float64


  6%|▌         | 1/17 [00:05<01:26,  5.39s/it]

Centering
Experting
S    0.600321
W    0.397473
P    0.002205
Name: 0, dtype: float64


 18%|█▊        | 3/17 [00:10<00:47,  3.41s/it]

...

 88%|████████▊ | 15/17 [01:16<00:11,  5.64s/it]

Centering
Experting
S    0.534181
W    0.464708
P    0.001111
Name: 0, dtype: float64


 94%|█████████▍| 16/17 [01:22<00:05,  5.70s/it]

Centering
Experting
W    0.541611
S    0.456589
P    0.001800
Name: 0, dtype: float64


100%|██████████| 17/17 [01:28<00:00,  5.21s/it]


In [ ]:
tmp = pd.DataFrame()
for dist in dists:
    tmp = pd.concat([tmp,dist],axis=1)

In [ ]:
import seaborn as sns
sns.kdeplot(tmp.T['P'].reset_index(drop=True),fill=True)
plt.xlabel("Proportion")
plt.title("Paradoxical Proportion Distribution Over Aging Cohort")
plt.savefig("p.jpg")

In [ ]:

sns.kdeplot(tmp.T['S'].reset_index(drop=True),color='green',fill=True)
plt.xlabel("Proportion")
plt.title("Slow Wave Proportion Distribution Over Aging Cohort")
plt.savefig("s.jpg")

In [ ]:

sns.kdeplot(tmp.T['W'].reset_index(drop=True),color='red',fill=True)
plt.xlabel("Proportion")
plt.title("Wakefulness Proportion Distribution Over Aging Cohort")
plt.savefig("w.jpg")

In [18]:
import matplotlib.pyplot as plt
import plotly.express as px
import numpy as np
import pandas as pd
from lib.utils import load_raw_by_path
from torch import nn
from torch.nn.functional import batch_norm,relu
from torch import nn
import numpy as np
from torch.nn.functional import softmax
import torch
import sqlite3
from sqlite3 import Error
import os
import pandas as pd
from tqdm import tqdm
EEG_1 = [1,8,14,15,16]
EEG_2 = [3,4,5,6,7,9,10,11,12,13,17]
i = 1
y_plot = np.array([])
raw = load_raw_by_path(f'aging/22-AGING-{i}.edf').get_data(picks=['EEG','EMG'])
if(i in EEG_1):
    eeg = raw[0]
elif(i in EEG_2):
    eeg = raw[1]
else:
    print("error")
del raw
eeg = eeg.reshape(-1,5000)
print("Centering")
eeg_centered = eeg - eeg.mean(axis=1).reshape(-1,1)
del eeg
std = eeg_centered.std(axis=1)
std[np.where(std==0)[0]] = 1
eeg_norm = eeg_centered/std.reshape(-1,1)
del eeg_centered
del std
X = torch.from_numpy(eeg_norm.reshape(-1,1,5000)).float()
del eeg_norm
print(X.isinf().any())
X = X[:,:,::10]
pred_expert = pd.read_csv(f'aging/{i}_pred.csv')
for pred in pred_expert.iloc:
    y_plot = np.concatenate([y_plot,np.array([pred[0]]*50)])

Centering
tensor(False)


In [6]:
df = pd.DataFrame([y_plot,X.flatten()[::10]]).T

In [11]:
X.flatten().shape

torch.Size([13378000])

In [7]:
fig = px.line(X.flatte.iloc[:50*90])
fig.show(renderer='browser')